In [1]:
using Distributed
# add processors if we don't have enough
# if length(workers()) < 6
# 	addprocs(6)
# end

@everywhere begin
	using Distributions
	using Plots
	using Turing
	using StatsPlots
end

In [2]:
@everywhere begin
	n_schools = 8
	y = [28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0] # estimated treatment effects
	σ = [15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0] # standard error of the effect estimate
end

In [3]:
@everywhere begin
	
	@model function schools(y, σ)
		μ ~ Normal(0, 5)
		τ ~ truncated(Cauchy(0, 5), lower = 0)
		θ ~ filldist(Normal(μ, τ), n_schools)
		for i in 1:n_schools
			y[i] ~ Normal(θ[i], σ[i])
		end
	end

	@model function school_reparam(y, σ)
		μ ~ Normal(0, 5)
		τ ~ truncated(Cauchy(0, 5), lower = 0)
		θ ~ filldist(Normal(0, 1), n_schools)
		# y .~ ForEach(μ .+ τ .* θ, σ)
		for i in 1:n_schools
			y[i] ~ Normal(μ + τ * θ[i], σ[i])
		end
	end
end


#### Inference with the No U-Turn Sampler (NUTS)

In [4]:
c1 = sample(schools(y, σ), NUTS(0.8),MCMCDistributed(), 2_000,12,n_adapts=1000)

┌ Warning: Only a single process available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC /home/mehdy/.julia/packages/AbstractMCMC/YrmkI/src/sample.jl:426
Sampling (1 processes)   0%|                            |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.8
└ @ Turing.Inference /home/mehdy/.julia/packages/Turing/r3Hmj/src/mcmc/hmc.jl:212
Sampling (1 processes)   8%|██▍                         |  ETA: 0:03:17
┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference /home/mehdy/.julia/packages/Turing/r3Hmj/src/mcmc/hmc.jl:212
Sampling (1 processes)  17%|████▋                       |  ETA: 0:01:44
┌ Info: Found initial step size
│   ϵ = 1.1875000000000002
└ @ Turing.Inference /home/mehdy/.julia/packages/Turing/r3Hmj/src/mcmc/hmc.jl:212
Sampling (1 processes)  25%|███████                     |  ETA: 0:01:10
┌ Info: Found initial step size
│   ϵ = 1.6
└ @ Turing.Inference /home/mehdy/.julia/packages/Turing/r3Hmj/src/mcmc/hmc.jl:212
Sampling (1 processes)  33%|██

CompositeException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] Normal(μ::ForwardDiff.Dual{ForwardDiff.Tag{DynamicPPL.DynamicPPLTag, Float64}, Float64, 10}, σ::Float64)
        @ Distributions ~/.julia/packages/Distributions/nmUhn/src/univariate/continuous/normal.jl:42
      [2] macro expansion
        @ ~/.julia/packages/DynamicPPL/DvdZw/src/compiler.jl:579 [inlined]
      [3] schools(__model__::DynamicPPL.Model{typeof(schools), (:y, :σ), (), (), Tuple{Vector{Float64}, Vector{Float64}}, Tuple{}, DynamicPPL.DefaultContext}, __varinfo__::DynamicPPL.TypedVarInfo{@NamedTuple{μ::DynamicPPL.Metadata{Dict{AbstractPPL.VarName{:μ, typeof(identity)}, Int64}, Vector{Normal{Float64}}, Vector{AbstractPPL.VarName{:μ, typeof(identity)}}, Vector{ForwardDiff.Dual{ForwardDiff.Tag{DynamicPPL.DynamicPPLTag, Float64}, Float64, 10}}, Vector{Set{DynamicPPL.Selector}}}, τ::DynamicPPL.Metadata{Dict{AbstractPPL.VarName{:τ, typeof(identity)}, Int64}, Vector{Truncate

In [ ]:
c2 = sample(school_reparam(y, σ), NUTS(0.8),MCMCDistributed(), 2_000,12,n_adapts=1000)

#### Emcee

In [ ]:
c3 = sample(schools(y, σ), Emcee(50),20_000)

In [ ]:
c4 = sample(school_reparam(y, σ), Emcee(50),20_000)

## Using MicroCanonical HMC

In [11]:
@everywhere begin
	using MicroCanonicalHMC
	n_adapts = 2_000 # adaptation steps
	tev = 0.001 # target energy variance
	mchmc = MCHMC(n_adapts, tev; adaptive = true)
end

In [ ]:
c5 = sample(schools(y, σ), externalsampler(mchmc),MCMCDistributed(), 10_000,6)

In [ ]:
c6 = sample(school_reparam(y, σ), externalsampler(mchmc),MCMCDistributed(), 10_000,6)

In [ ]:
plot(c1)

In [ ]:
plot(c2)

In [ ]:
plot(c3)

In [ ]:
plot(c4)

In [ ]:
plot(c5)

In [ ]:
plot(c6)